In [1]:
import os
import random
from typing import Annotated

from dotenv import load_dotenv

from openai import AsyncOpenAI
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

# Load environment variables
load_dotenv()

# Define a Coding Plugin
class CodingPlugin:
    """Provides coding help and generates code examples."""

    @kernel_function(description="Generates Python code examples.")
    def generate_python_example(self, topic: Annotated[str, "The coding topic or task"]) -> Annotated[str, "Python code example for the given topic"]:
        examples = {
            "fibonacci": "def fibonacci(n):\n    a, b = 0, 1\n    for _ in range(n):\n        print(a)\n        a, b = b, a + b",
            "factorial": "def factorial(n):\n    return 1 if n == 0 else n * factorial(n - 1)",
            "list comprehension": "[x**2 for x in range(10)]",
            "api request": "import requests\nresponse = requests.get('https://api.example.com')\nprint(response.json())",
        }
        return examples.get(topic.lower(), "# No example found for this topic.")

# Set up OpenAI client
client = AsyncOpenAI(
    api_key=os.getenv("GITHUB_TOKEN"),
    base_url="https://models.inference.ai.azure.com/",
)

# Set up the OpenAI chat completion service
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",  # Use your actual deployed model or Azure deployment name
    async_client=client,
)

# Set up the Coding Assistant Agent
agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[CodingPlugin()],
    name="CodingAgent",
    instructions="You are a helpful coding assistant that generates and explains code examples and concepts in Python and other programming languages.",
)

# Async main function
async def main():
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Show me a list comprehension in Python.",
        "Can you explain how to use requests in Python?",
    ]

    for user_input in user_inputs:
        print(f"\n# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    await thread.delete() if thread else None

# Run the agent
await main()



# User: Show me a list comprehension in Python.

# CodingAgent: Here is an example of a list comprehension in Python:

```python
# Example: Create a list of squares for numbers from 0 to 9

squares = [x**2 for x in range(10)]
print(squares)
```

### Explanation:
- `range(10)` generates numbers from 0 to 9.
- The expression `x**2` computes the square of each number.
- The list comprehension `[x**2 for x in range(10)]` creates a new list containing the squares of the numbers in the specified range. 

When you run the above code, it will output:

```
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
```

This is a concise way to create lists in Python.

# User: Can you explain how to use requests in Python?

# CodingAgent: Certainly! The `requests` library in Python is a powerful tool for making HTTP requests. Here's a brief explanation along with examples of how to use it.

### Installation
First, you need to install the `requests` library if you haven't already. You can do this using pip:

```bash
